In [24]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import collections

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import datetime
import pandas as pd

torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)

In [25]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [26]:
from torchvision import datasets, transforms
data_path = '../../data-unversioned/p1ch7/'
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

Files already downloaded and verified


In [27]:
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

Files already downloaded and verified


In [28]:
label_map = {0: 0, 2: 1}
class_names = ['airplane', 'bird']
cifar2 = [(img, label_map[label])
          for img, label in cifar10
          if label in [0, 2]]
cifar2_val = [(img, label_map[label])
              for img, label in cifar10_val
              if label in [0, 2]]

## 1.1

In [42]:
model = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=5, padding=1),
            nn.Tanh(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 8, kernel_size=5, padding=1),
            nn.Tanh(),
            nn.MaxPool2d(2),
            # ... <1>
            nn.Flatten(), 
            nn.Linear(8 * 8 * 8, 32),
            nn.Tanh(),
            nn.Linear(32, 2))

In [52]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=5, padding=2)
        self.act1 = nn.Tanh()
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=5, padding=2)
        self.act2 = nn.Tanh()
        self.pool2 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        self.act3 = nn.Tanh()
        self.fc2 = nn.Linear(32, 2)

    def forward(self, x):
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        out = out.view(-1, 8 * 8 * 8)
        out = self.act3(self.fc1(out))
        out = self.fc2(out)
        return out
    
model = Net()

👆

注意`padding`, 加边

In [53]:
numel_list = [p.numel() for p in model.parameters()]
sum(numel_list), numel_list

(20906, [1200, 16, 3200, 8, 16384, 32, 64, 2])

变大了, lol

---

## 1.2

In [54]:
device = (torch.device('cuda') if torch.cuda.is_available()
          else torch.device('cpu'))

print(f"Training on device: {device}.")

Training on device: cuda.


In [55]:
cifar2_gpu = [(
    img.to(device=device),
    torch.tensor(label).to(device=device)
    )
    for img, label in cifar2
]

cifar2_val_gpu = [(
    img.to(device=device),
    torch.tensor(label).to(device=device)
    )
    for img, label in cifar2_val
]

In [56]:
class ResBlock(nn.Module):
    def __init__(self, n_chans):
        super(ResBlock, self).__init__()
        self.conv = nn.Conv2d(n_chans, n_chans, kernel_size=3,
                              padding=1, bias=False)
        self.batch_norm = nn.BatchNorm2d(num_features=n_chans)
        torch.nn.init.kaiming_normal_(self.conv.weight,
                                      nonlinearity='relu')
        torch.nn.init.constant_(self.batch_norm.weight, 0.5)
        torch.nn.init.zeros_(self.batch_norm.bias)

    def forward(self, x):
        out = self.conv(x)
        out = self.batch_norm(out)
        out = torch.relu(out)
        return out + x

In [57]:
class NetResDeep(nn.Module):
    def __init__(self, n_chans1=32, n_blocks=10):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.resblocks = nn.Sequential(
            *(n_blocks * [ResBlock(n_chans=n_chans1)]))
        self.fc1 = nn.Linear(8 * 8 * n_chans1, 32)
        self.fc2 = nn.Linear(32, 2)
        
    def forward(self, x):
        out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
        out = self.resblocks(out)
        out = F.max_pool2d(out, 2)
        out = out.view(-1, 8 * 8 * self.n_chans1)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out

In [58]:
def training_loop(n_epochs, optimizer, model, loss_fn,
                        train_loader, val_loader):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        loss_val = 0.0
        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)
            labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            l2_lambda = 0.001
            l2_norm = sum(p.pow(2.0).sum()
                          for p in model.parameters())
            loss = loss + l2_lambda * l2_norm

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            loss_train += loss.item()

        for imgs_val, labels_val in val_loader:
            imgs_val = imgs_val.to(device=device)
            labels_val = labels_val.to(device=device)
            outputs_val= model(imgs_val)
            loss_val = loss_fn(outputs_val, labels_val)
            loss_val += loss_val.item()
        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}, Validation loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader),
                loss_val / len(val_loader)))


In [59]:
def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():
            for imgs, labels in loader:
                imgs = imgs.to(device=device)
                labels = labels.to(device=device)
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1)
                total += labels.shape[0]
                correct += int((predicted == labels).sum())

        print("Accuracy {}: {:.2f}".format(name , correct / total))

In [60]:
train_loader_gpu = torch.utils.data.DataLoader(cifar2_gpu, batch_size=64, shuffle=True) 
val_loader_gpu = torch.utils.data.DataLoader(cifar2_val_gpu, batch_size=64, shuffle=False)

In [61]:
model = NetResDeep(n_chans1=32).to(device=device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader_gpu,
    val_loader = val_loader_gpu
)

2024-02-03 10:50:27.277679 Epoch 1, Training loss 0.6104012391749462, Validation loss 0.03489741310477257
2024-02-03 10:50:50.351137 Epoch 10, Training loss 0.3131644684987463, Validation loss 0.0203187707811594
2024-02-03 10:51:18.850487 Epoch 20, Training loss 0.21637335936924454, Validation loss 0.025120534002780914
2024-02-03 10:51:47.144429 Epoch 30, Training loss 0.14033328077405882, Validation loss 0.02068961225450039
2024-02-03 10:52:13.026277 Epoch 40, Training loss 0.13399474669224137, Validation loss 0.039109956473112106
2024-02-03 10:52:40.384124 Epoch 50, Training loss 0.11893341486241407, Validation loss 0.031134864315390587
2024-02-03 10:53:07.490842 Epoch 60, Training loss 0.2603666202467718, Validation loss 0.03549351915717125
2024-02-03 10:53:35.664692 Epoch 70, Training loss 0.08294183571057714, Validation loss 0.06914859265089035
2024-02-03 10:54:03.030088 Epoch 80, Training loss 0.1255132863476018, Validation loss 0.0520419143140316
2024-02-03 10:54:29.204309 Epoch

In [62]:
temp = validate(model, train_loader_gpu, val_loader_gpu)

Accuracy train: 1.00
Accuracy val: 0.91


感觉`overfit`, 换单纯的`resnet`

In [63]:
class NetRes(nn.Module):
    def __init__(self, n_chans1=32):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(n_chans1, n_chans1 // 2, kernel_size=3,
                               padding=1)
        self.conv3 = nn.Conv2d(n_chans1 // 2, n_chans1 // 2,
                               kernel_size=3, padding=1)
        self.fc1 = nn.Linear(4 * 4 * n_chans1 // 2, 32)
        self.fc2 = nn.Linear(32, 2)
        
    def forward(self, x):
        out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
        out = F.max_pool2d(torch.relu(self.conv2(out)), 2)
        out1 = out
        out = F.max_pool2d(torch.relu(self.conv3(out)) + out1, 2)
        out = out.view(-1, 4 * 4 * self.n_chans1 // 2)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out

In [64]:
model = NetRes(n_chans1=32).to(device=device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader_gpu,
    val_loader = val_loader_gpu
)

2024-02-03 10:54:58.191926 Epoch 1, Training loss 0.6834989850688132, Validation loss 0.034787606447935104
2024-02-03 10:55:05.780613 Epoch 10, Training loss 0.369570419287226, Validation loss 0.022408757358789444
2024-02-03 10:55:14.216617 Epoch 20, Training loss 0.32370744190018647, Validation loss 0.027914611622691154
2024-02-03 10:55:22.826954 Epoch 30, Training loss 0.2901740530683736, Validation loss 0.021025696769356728
2024-02-03 10:55:31.080546 Epoch 40, Training loss 0.268691055144474, Validation loss 0.024957172572612762
2024-02-03 10:55:39.185375 Epoch 50, Training loss 0.2500746067921827, Validation loss 0.02278066985309124
2024-02-03 10:55:47.060876 Epoch 60, Training loss 0.22987363387824625, Validation loss 0.018666250631213188
2024-02-03 10:55:54.617541 Epoch 70, Training loss 0.21082317757948188, Validation loss 0.023035326972603798
2024-02-03 10:56:02.990660 Epoch 80, Training loss 0.19486768130853677, Validation loss 0.016683857887983322
2024-02-03 10:56:11.999562 E

In [65]:
temp = validate(model, train_loader_gpu, val_loader_gpu)

Accuracy train: 0.96
Accuracy val: 0.90


## 1.3

In [66]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=(3, 1), padding=1)
        self.act1 = nn.Tanh()
        self.pool1 = nn.MaxPool2d(2) # 16x17
        self.conv2 = nn.Conv2d(16, 8, kernel_size=(3, 1), padding=1)
        self.act2 = nn.Tanh() # 16x18
        self.pool2 = nn.MaxPool2d(2) # 8x9
        self.fc1 = nn.Linear(8 * 9 * 8, 32)
        self.act3 = nn.Tanh()
        self.fc2 = nn.Linear(32, 2)

    def forward(self, x):
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        out = out.view(-1, 8 * 8 * 8)
        out = self.act3(self.fc1(out))
        out = self.fc2(out)
        return out
    
model = Net()

In [67]:
model = NetRes(n_chans1=32).to(device=device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader_gpu,
    val_loader = val_loader_gpu
)

2024-02-03 10:58:51.023968 Epoch 1, Training loss 0.6668332179260862, Validation loss 0.035441283136606216
2024-02-03 10:58:57.827627 Epoch 10, Training loss 0.3652876191268301, Validation loss 0.019229363650083542
2024-02-03 10:59:05.291140 Epoch 20, Training loss 0.3238427865846901, Validation loss 0.01762267015874386
2024-02-03 10:59:12.925929 Epoch 30, Training loss 0.29261245469378816, Validation loss 0.02179892174899578
2024-02-03 10:59:20.876394 Epoch 40, Training loss 0.2734147717428815, Validation loss 0.016872115433216095
2024-02-03 10:59:28.749108 Epoch 50, Training loss 0.2448071079554072, Validation loss 0.0215920340269804
2024-02-03 10:59:36.440292 Epoch 60, Training loss 0.2256851297370188, Validation loss 0.020621882751584053
2024-02-03 10:59:44.318465 Epoch 70, Training loss 0.20925082157182087, Validation loss 0.0212523452937603
2024-02-03 10:59:52.016799 Epoch 80, Training loss 0.18875115753928567, Validation loss 0.019672119989991188
2024-02-03 10:59:59.143626 Epoch

In [68]:
temp = validate(model, train_loader_gpu, val_loader_gpu)

Accuracy train: 0.96
Accuracy val: 0.89


## 1.4

In [98]:
# 懒得跑前面去运行了, 我直接把一个莫模型贴过来重新算

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=5, padding=2)
        self.act1 = nn.Tanh()
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=5, padding=2)
        self.act2 = nn.Tanh()
        self.pool2 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        self.act3 = nn.Tanh()
        self.fc2 = nn.Linear(32, 2)

    def forward(self, x):
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        out = out.view(-1, 8 * 8 * 8)
        out = self.act3(self.fc1(out))
        out = self.fc2(out)
        return out
    
model = Net()

class NetRes(nn.Module):
    def __init__(self, n_chans1=32):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(n_chans1, n_chans1 // 2, kernel_size=3,
                               padding=1)
        self.conv3 = nn.Conv2d(n_chans1 // 2, n_chans1 // 2,
                               kernel_size=3, padding=1)
        self.fc1 = nn.Linear(4 * 4 * n_chans1 // 2, 32)
        self.fc2 = nn.Linear(32, 2)
        
    def forward(self, x):
        out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
        out = F.max_pool2d(torch.relu(self.conv2(out)), 2)
        out1 = out
        out = F.max_pool2d(torch.relu(self.conv3(out)) + out1, 2)
        out = out.view(-1, 4 * 4 * self.n_chans1 // 2)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out
    
model = NetRes(n_chans1=32).to(device=device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 50,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader_gpu,
    val_loader = val_loader_gpu
)

temp = validate(model, train_loader_gpu, val_loader_gpu)

2024-02-03 11:43:10.695515 Epoch 1, Training loss 0.6379847699289869, Validation loss 0.0316740944981575
2024-02-03 11:43:17.135070 Epoch 10, Training loss 0.3563068223417185, Validation loss 0.025845563039183617
2024-02-03 11:43:25.129758 Epoch 20, Training loss 0.31497339780922906, Validation loss 0.019146736711263657
2024-02-03 11:43:32.634904 Epoch 30, Training loss 0.29031328173579685, Validation loss 0.019030680879950523
2024-02-03 11:43:40.559151 Epoch 40, Training loss 0.2704985573603089, Validation loss 0.02380673959851265
2024-02-03 11:43:48.335279 Epoch 50, Training loss 0.24516844198961926, Validation loss 0.012765792198479176
Accuracy train: 0.92
Accuracy val: 0.90


In [71]:
cifar2_outside = [(
    img.to(device=device),
    torch.tensor(label).to(device=device)
    )
    for img, label in cifar10
    if label not in [0, 2]]

In [99]:
num_wrong = 0

for i, (img, label) in enumerate(cifar2_outside):
    if i+1 == 1:
        print("progression: 0 %")
    elif (i+1) % 5000 == 0:
        print("progression: {} %".format((i+1)/len(cifar2_outside) * 100))
    img = img.unsqueeze(0).to(device=device)
    m = nn.Softmax(dim=1)
    prediction = m(model(img))

    if prediction[0][0] > 0.95 or prediction[0][1] < 0.05:
        # print(i, label, prediction)
        num_wrong += 1
        prediction_pd = pd.DataFrame([[label.cpu().item(), prediction.cpu().detach().numpy()[0]]], columns=['label', 'prediction'])
        prediction_pd.to_csv('prediction.csv', mode='a')

progression: 0 %
progression: 12.5 %
progression: 25.0 %
progression: 37.5 %
progression: 50.0 %
progression: 62.5 %
progression: 75.0 %
progression: 87.5 %
progression: 100.0 %


In [100]:
num_wrong, num_wrong / len(cifar2_outside)

(6574, 0.16435)

---

In [103]:
class ResBlock(nn.Module):
    def __init__(self, n_chans):
        super(ResBlock, self).__init__()
        self.conv = nn.Conv2d(n_chans, n_chans, kernel_size=3,
                              padding=1, bias=False)
        self.batch_norm = nn.BatchNorm2d(num_features=n_chans)
        torch.nn.init.kaiming_normal_(self.conv.weight,
                                      nonlinearity='relu')
        torch.nn.init.constant_(self.batch_norm.weight, 0.5)
        torch.nn.init.zeros_(self.batch_norm.bias)

    def forward(self, x):
        out = self.conv(x)
        out = self.batch_norm(out)
        out = torch.relu(out)
        return out + x
    
class NetResDeep(nn.Module):
    def __init__(self, n_chans1=32, n_blocks=10):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.resblocks = nn.Sequential(
            *(n_blocks * [ResBlock(n_chans=n_chans1)]))
        self.fc1 = nn.Linear(8 * 8 * n_chans1, 32)
        self.fc2 = nn.Linear(32, 2)
        
    def forward(self, x):
        out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
        out = self.resblocks(out)
        out = F.max_pool2d(out, 2)
        out = out.view(-1, 8 * 8 * self.n_chans1)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out

model = NetResDeep(n_chans1=32).to(device=device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 50,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader_gpu,
    val_loader = val_loader_gpu
)

temp = validate(model, train_loader_gpu, val_loader_gpu)

print(temp)

num_wrong = 0

for i, (img, label) in enumerate(cifar2_outside):
    if i+1 == 1:
        print("progression: 0 %")
    elif (i+1) % 5000 == 0:
        print("progression: {} %".format((i+1)/len(cifar2_outside) * 100))
    img = img.unsqueeze(0).to(device=device)
    m = nn.Softmax(dim=1)
    prediction = m(model(img))

    if prediction[0][0] > 0.95 or prediction[0][1] < 0.05:
        # print(i, label, prediction)
        num_wrong += 1
        prediction_pd = pd.DataFrame([[label.cpu().item(), prediction.cpu().detach().numpy()[0]]])
        prediction_pd.to_csv('prediction.csv', mode='a')

2024-02-03 11:56:09.179214 Epoch 1, Training loss 0.6051068778630275, Validation loss 0.02200504019856453
2024-02-03 11:56:30.712786 Epoch 10, Training loss 0.32701538920782175, Validation loss 0.015018164180219173
2024-02-03 11:56:54.747690 Epoch 20, Training loss 0.23317246433276279, Validation loss 0.029492247849702835
2024-02-03 11:57:18.833585 Epoch 30, Training loss 0.1877649053456677, Validation loss 0.02070627547800541
2024-02-03 11:57:43.054405 Epoch 40, Training loss 0.11485524988098508, Validation loss 0.02831980399787426
2024-02-03 11:58:09.394786 Epoch 50, Training loss 0.08638301159545875, Validation loss 0.039085838943719864
Accuracy train: 1.00
Accuracy val: 0.91
None
progression: 0 %
progression: 12.5 %
progression: 25.0 %
progression: 37.5 %
progression: 50.0 %
progression: 62.5 %
progression: 75.0 %
progression: 87.5 %
progression: 100.0 %


In [104]:
num_wrong, num_wrong / len(cifar2_outside)

(11042, 0.27605)

In [ ]:
# for i, (img, label) in enumerate(cifar2_outside):
#     outputs = model(img.unsqueeze(0))
#     _, predicted = torch.max(outputs, dim=1)
#     if predicted != label:
#         print(f"第{i}张图片被错误分类为{class_names[predicted]}, 实际上是{class_names[label]}")